In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/big-data-analytics-certification-kr-2023-5th/sample_submission.csv
/kaggle/input/big-data-analytics-certification-kr-2023-5th/train.csv
/kaggle/input/big-data-analytics-certification-kr-2023-5th/test.csv


In [138]:
X_train = pd.read_csv('/kaggle/input/big-data-analytics-certification-kr-2023-5th/train.csv')
X_test = pd.read_csv('/kaggle/input/big-data-analytics-certification-kr-2023-5th/test.csv')

In [139]:
display(X_train)
display(X_test)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,EcoSport,2017,10298,Manual,25013,Petrol,150,53.3,1.0
1,Focus,2016,10491,Manual,30970,Diesel,0,74.3,1.5
2,S-MAX,2017,13498,Manual,60200,Diesel,150,56.5,2.0
3,Fiesta,2018,10600,Manual,27380,Petrol,145,56.5,1.0
4,Fiesta,2018,11000,Manual,7724,Petrol,145,65.7,1.0
...,...,...,...,...,...,...,...,...,...
3754,Fiesta,2017,10495,Automatic,31597,Petrol,0,57.7,1.0
3755,Kuga,2019,17991,Manual,3961,Diesel,150,45.6,1.5
3756,Ka+,2017,6830,Manual,16207,Petrol,145,57.7,1.2
3757,Fiesta,2018,11500,Manual,9487,Petrol,150,65.7,1.0


,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,S-MAX,2016,Manual,23905,Diesel,125,56.5,2.0
1,Fiesta,2018,Manual,16895,Petrol,145,40.3,1.5
2,Kuga,2017,Manual,12109,Petrol,165,45.6,1.5
3,Kuga,2018,Semi-Auto,13940,Diesel,145,58.9,1.5
4,Fiesta,2013,Manual,31690,Petrol,30,54.3,1.2
...,...,...,...,...,...,...,...,...
1612,Fiesta,2018,Manual,15950,Petrol,145,65.7,1.0
1613,Focus,2018,Manual,18003,Diesel,145,67.3,2.0
1614,Grand C-MAX,2016,Manual,36489,Diesel,30,64.2,1.5
1615,Kuga,2017,Manual,38969,Diesel,125,60.1,2.0


In [140]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3759 entries, 0 to 3758
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3759 non-null   object 
 1   year          3759 non-null   int64  
 2   price         3759 non-null   int64  
 3   transmission  3759 non-null   object 
 4   mileage       3759 non-null   int64  
 5   fuelType      3759 non-null   object 
 6   tax           3759 non-null   int64  
 7   mpg           3759 non-null   float64
 8   engineSize    3759 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 264.4+ KB


In [141]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1617 entries, 0 to 1616
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1617 non-null   object 
 1   year          1617 non-null   int64  
 2   transmission  1617 non-null   object 
 3   mileage       1617 non-null   int64  
 4   fuelType      1617 non-null   object 
 5   tax           1617 non-null   int64  
 6   mpg           1617 non-null   float64
 7   engineSize    1617 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 101.2+ KB


In [142]:
X_train.isnull().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [143]:
X_test.isnull().sum()

model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [144]:
y_train = X_train['price']
X_train_drop = X_train.drop(columns='price')
model_name = X_test['model']

In [145]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

n_col = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
v_col = ['transmission', 'fuelType', 'model']

ss = StandardScaler()
le = LabelEncoder()

ss.fit_transform(X_train_drop[n_col])
ss.transform(X_test[n_col])

for i in v_col:
    X_train_drop[i] = le.fit_transform(X_train_drop[i])
    X_test[i] = le.transform(X_test[i])

In [146]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

X_tr, X_val, y_tr, y_val = train_test_split(X_train_drop, y_train, test_size=0.2, random_state=41)

rf = RandomForestRegressor()
xgb = XGBRegressor()

rf.fit(X_tr, y_tr)
print(rmse(y_tr, rf.predict(X_tr)))
print(rmse(y_val, rf.predict(X_val)))

xgb.fit(X_tr, y_tr)
print(rmse(y_tr, xgb.predict(X_tr)))
print(rmse(y_val, xgb.predict(X_val)))

523.4845184254006
1311.2345672046245
630.5839103687294
1188.2254474198908


In [153]:
pd.DataFrame({"id": X_test.index, "price":rf.predict(X_test_dup)}).to_csv('000000000.csv', index=False)
# 실제 시험 답안 제출 양식
# pd.DataFrame({"pred":rf.predict(X_test)}).to_csv('000000000.csv', index=False)

In [154]:
result = pd.read_csv('000000000.csv')
result

,id,price
0,0,15537.10
1,1,15249.18
2,2,14600.94
3,3,18404.13
4,4,6291.40
...,...,...
1612,1612,11619.89
1613,1613,17943.51
1614,1614,9727.20
1615,1615,13977.77
